## SQL mit DuckDB

### Installation
Auf https://duckdb.org/#quickinstall lässt sich überprüfen, welches die neueste Version ist. Wir arbeiten hier mit 0.7.1.
- jupysql: Konvertiere eine Notebook Zelle in eine SQL Zelle via a %sql and %%sql magics
- duckdb-engine: duckdb Driver, baut Verbindung zu einer DuckDB Datenbank auf

In [ ]:
#!pip install duckdb==0.7.1

In [ ]:
#!pip install jupysql    
#!pip install duckdb-engine 
#!pip install wget

In [3]:
# Download JSON Dataset
import wget
filename = wget.download('https://vega.github.io/vega-datasets/data/cars.json')

In [4]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

/tmp/ipykernel_15364/3250854491.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Deploy Flask apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [5]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [6]:
%sql duckdb:///memory
# %sql duckdb:///path/to/file.db

Einzeilige SQL Anfragen können mit %sql am Beginn der Zeile abgesetzt werden. Die Ergebnisse der Anfrage werden mittels eines Pandas Dataframe angezeigt.

In [7]:
%sql SELECT 'Off and flying!' as a_duckdb_column

,a_duckdb_column
0,Off and flying!


In [8]:
%sql CREATE TABLE cars AS SELECT * FROM read_json_auto('cars.json')

,Count
0,406


*Aufgabe*: Geben Sie die ersten 5 Zeilen aus

In [9]:
%sql SELECT * FROM cars LIMIT 5

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
0,chevrolet chevelle malibu,18.0,8,307.0,130,3504,12.0,1970-01-01,USA
1,buick skylark 320,15.0,8,350.0,165,3693,11.5,1970-01-01,USA
2,plymouth satellite,18.0,8,318.0,150,3436,11.0,1970-01-01,USA
3,amc rebel sst,16.0,8,304.0,150,3433,12.0,1970-01-01,USA
4,ford torino,17.0,8,302.0,140,3449,10.5,1970-01-01,USA


In [12]:
%sql SELECT DISTINCT year FROM cars 

,Year
0,1970-01-01
1,1971-01-01
2,1972-01-01
3,1973-01-01
4,1974-01-01
5,1975-01-01
6,1976-01-01
7,1977-01-01
8,1978-01-01
9,1979-01-01


Aufgabe: Wie viele Einträge gibt es pro Land? 

In [15]:
%sql SELECT origin, COUNT(*) as count FROM cars GROUP BY origin

,Origin,count
0,USA,254
1,Europe,73
2,Japan,79


Frage: Was ist der Unterschied zwischen count und count distinct? 

_Antwort_: `COUNT` zählt einfach alles und `DISTINCT` zählt nur die einzigartigen Werte.

Einlesen einer Datenbankabfrage in ein Pandas DataFrame. Mit diesem kann anschließend ganz normal weitergearbeitet werden, zum Beispiel kann die Anfrage visualisiert werden.

In [16]:
%sql my_df << SELECT Origin, count (*) as quantity FROM cars GROUP BY 1

In [17]:
import altair as alt
alt.Chart(my_df).mark_bar().encode(         # type: ignore
    x='Origin',
    y='quantity'
)

alt.Chart(...)

#### Aggregationen

In [18]:
%%sql
SELECT COUNT(*) AS '#rows',
       AVG(weight_in_lbs) AS 'avg',
       SUM(weight_in_lbs) AS 'sum',
       MAX(weight_in_lbs) AS 'max'
FROM cars 
       

,#rows,avg,sum,max
0,406,2979.413793,1209642.0,5140


Aufgabe: Geben Sie alle Gewichte als String geordnet nach der Zylindergröße (c.cylinders DESC) als "all weights" aus. Nutzen Sie die Funktion string_agg(). 

#### Pivoting

In [ ]:
%%sql
SELECT AVG(c.weight_in_lbs) FILTER(WHERE c.origin='USA') AS 'avg'
FROM cars AS c

Aufgabe: Geben Sie die Durchschnittsgewichte für die USA, Japan und Europa aus

#### Grouping

In [ ]:
%%sql
SELECT origin, ROUND(AVG(c.weight_in_lbs),2) AS 'avg'
FROM cars AS c
GROUP BY c.origin

Aufgabe: Erweitern Sie die obige Anfrage, so dass nur der Durchschnitt der europäischen Länder ausgegeben wird. 

#### Subqueries 
Wie oft kommen bestimmte Attribute in einem Datensatz vor? 

In [ ]:
%%sql 
SELECT nb_new_cars, count(*) as num_years
FROM (
    SELECT year, count(name) AS nb_new_cars
    FROM cars AS c
    GROUP BY 1
) a
GROUP BY 1

In [ ]:
%%sql
SELECT year, count(name) AS nb_new_cars
FROM cars AS c
GROUP BY 1

#### Binning

In [ ]:
%%sql
SELECT 
case when horsepower <= 70 then '0: 0 - 70'
     when horsepower <= 140 then '1: 70 - 140'
     when horsepower <= 210 then '2: 140 - 210'
     else '3: 210+' end as horsepower_bin
,count(name) as cars
FROM cars
GROUP BY 1
order by horsepower_bin
;

In [ ]:
%%sql
SELECT min(horsepower), max(horsepower) FROM cars

### Weitere Datenformate
DuckDB’s httpfs extension allows parquet and csv files to be queried remotely over http. These examples query a parquet file that contains historical taxi data from NYC. Using the parquet format allows DuckDB to only pull the rows and columns into memory that are needed rather than download the entire file. DuckDB can be used to process local parquet files as well, which may be desirable if querying the entire parquet file, or running multiple queries that require large subsets of the file.

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;

In [ ]:
%sqlplot boxplot --table https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet --column trip_distance